# Predictions statistics

Analysis of the predictions made for a certain datasplit (usually test).

**Note:** To run this notebook you previously have to run and save the predictions on a test dataset **with ground-truth values**. See the Datasplit section in [3.0 notebook](./3.0-Computing_predictions.ipynb#Predicting-a-datasplit-txt-file) for reference.


In [4]:
def Average(lst): 
    return sum(lst) / len(lst) 

In [5]:
import os
import json
import numpy as np

from imgclas.data_utils import load_image
from imgclas import paths, plot_utils
from imgclas import test_utils
import sklearn.metrics as metrics

import warnings
warnings.filterwarnings("ignore")

# User parameters to set
TIMESTAMP = '2023-04-30_132931'       # timestamp of the model              
SPLIT_NAME = 'test'                   # dataset split to predict
MODEL_NAME = 'final_model.h5'         # model to use to make the mediction
FOLDS = ["Fold-0","Fold-1","Fold-2","Fold-3","Fold-4"]

folds_mae = list()
folds_std = list()
folds_mse = list()
folds_rmse = list()
folds_r2 = list()

# Set the timestamp
paths.timestamp = TIMESTAMP

for FOLD in FOLDS:

    # Load back the predictions
    pred_path = os.path.join(paths.get_predictions_dir()+'/'+FOLD, '{}+{}.json'.format(MODEL_NAME, SPLIT_NAME))
    with open(pred_path) as f:
        pred_dict = json.load(f)

    y, yhat = np.array(pred_dict['true_value']), np.array(pred_dict['pred_value'])

    folds_mae.append(metrics.mean_absolute_error(y, yhat))
    folds_std.append(np.std(np.abs(yhat-y)))
    mse=metrics.mean_squared_error(y, yhat)
    folds_mse.append(mse)
    folds_rmse.append(np.sqrt(mse))# or mse**(0.5)  
    folds_r2.append(metrics.r2_score(y,yhat))

print("MAE:", Average(folds_mae))
print("MAE:",folds_mae)
#print("STD:",folds_std)
print("STD:",Average(folds_std))
print("MSE:", Average(folds_mse))
print("RMSE:", Average(folds_rmse))
print("R-Squared:", Average(folds_r2))

MAE: 20.569730957209398
MAE: [21.800057456255143, 18.22611904732975, 19.514024613816062, 18.10000053452857, 25.20845313411748]
STD: 17.400416743685344
MSE: 738.1106652351158
RMSE: 26.94468634290512
R-Squared: 0.6735727225880812
